# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

**Шаги для выбора локации:**
- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

**Условия задачи:**
- Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
- При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
- Бюджет на разработку скважин в регионе — 10 млрд рублей.
- При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
- После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

**Описание данных:**
- id — уникальный идентификатор скважины;
- f0, f1, f2 — три признака точек;
- product — объём запасов в скважине (тыс. баррелей).

## Загрузка и подготовка данных

In [1]:
# pip install sweetviz

In [2]:
# импортируем библиотеки
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from joblib import dump, load
import sweetviz as sv

In [3]:
# загрузим данные
geo_data_0 = pd.read_csv('datasets/geo_data_0.csv')
geo_data_1 = pd.read_csv('datasets/geo_data_1.csv')
geo_data_2 = pd.read_csv('datasets/geo_data_2.csv')

for data, name in zip([geo_data_0, geo_data_1, geo_data_2], 
                ["geo_data_0", "geo_data_1", "geo_data_2"]):
    print("Регион {}".format(name))
    display(data.head(3))
    print("Количество дубликатов {}, пропущенных значений {}"
         .format(data.duplicated().sum(), data.isna().sum().sum()))

Регион geo_data_0


,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647


Количество дубликатов 0, пропущенных значений 0
Регион geo_data_1


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305


Количество дубликатов 0, пропущенных значений 0
Регион geo_data_2


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910


Количество дубликатов 0, пропущенных значений 0


- Проведем исследовательский анализ данных с помощью пакета sweetviz.

In [ ]:
report_0 = sv.analyze([geo_data_0[["f0", "f1", "f2", "product"]], "geo_data_0"], target_feat="product")
report_0.show_html("sweetviz/geo_data_0.html")
report_0.show_notebook()

In [ ]:
report_1 = sv.analyze([geo_data_1[["f0", "f1", "f2", "product"]], "geo_data_1"], target_feat="product")
report_1.show_html("sweetviz/geo_data_1.html")
report_1.show_notebook()

In [ ]:
report_2 = sv.analyze([geo_data_2[["f0", "f1", "f2", "product"]], "geo_data_2"], target_feat="product")
report_2.show_html("sweetviz/geo_data_2.html")
report_2.show_notebook()

Исследуемые данные обладают следующими особенностями:
- не наблюдается пропущенных значений, дубликатов и выбросов; 
- распределение целевого признака (запас сырья) в регионах 0 и 2 близко к нормальному, а в регионе 1 больше соответствует равномерному, но с увеличенным содержанием скважин с минимальным и максимальным запасом сырья;
- в регионе 0 целевой признак коррелирует со всеми 3 признаками (f0, f1, f2) и имеет наибольшую корреляцию с f2;
- в регионах 1 и 2 целевой признак коррелируе главным образов с признаком f2 и намного слабее с остальными, причем в регионе 1 данная корреляция является очень сильной, в в регионе 2 находится на среднем уровне.

## Обучение и проверка модели

- Разобъем данные для каждого региона на обущающую и валидационную выборку в соотношении 75/25.

Создадим функцию, которая разбивает данные на необходимые для анализа признаки, и целевой признак, после чего делит их на обучающую и валидационную выборку.

In [7]:
def data_split(geo_data, features_drop=["id", "product"], target="product",
                  test_size=.25, random_state=12345):
    """
    Функция принимает на вход датафрейм с геоданными.
    Делит его на признаки (f0, f1, f2) и целевой признак (product), 
    после этого разделяет на обучающую и валидационную выборки в соотношении 75/25.
    На выходе выдает словарь с признаками и целевым признаком получившихся выборок.
    """
    features = geo_data.drop(features_drop, axis=1)
    target = geo_data[target]
    
    features_train, features_valid, target_train, target_valid = \
                    train_test_split(features, target, test_size=test_size, random_state=random_state)
    
    return {"features_train": features_train, "target_train": target_train, 
            "features_valid": features_valid, "target_valid": target_valid}

Проверим работу функции и разобъем наши геоданные.

In [8]:
for data, name in zip([geo_data_0, geo_data_1, geo_data_2], 
                      ["geo_data_0", "geo_data_1", "geo_data_2"]):
    print("Датафрейм {}\
         \nРазмеры обучающей выборки: features{}, target{}\
         \nРазмеры валидационной выборки: features{}, target{}\n{}"
         .format(name,
                 data_split(data)["features_train"].shape,
                 data_split(data)["target_train"].shape,
                 data_split(data)["features_valid"].shape,
                 data_split(data)["target_valid"].shape,
                 " "))

Датафрейм geo_data_0         
Размеры обучающей выборки: features(75000, 3), target(75000,)         
Размеры валидационной выборки: features(25000, 3), target(25000,)
 
Датафрейм geo_data_1         
Размеры обучающей выборки: features(75000, 3), target(75000,)         
Размеры валидационной выборки: features(25000, 3), target(25000,)
 
Датафрейм geo_data_2         
Размеры обучающей выборки: features(75000, 3), target(75000,)         
Размеры валидационной выборки: features(25000, 3), target(25000,)
 


Выборки разделились в нужной нам пропорции.

Найдем модели машинного обучения с наименьшим значением RMSE для валидационной выборки каждого региона. Будем исследовать разные типы линейных моделей:
- стандартную линейную регрессию (LinearRegression)
- линейную регрессию с L2-регуляризацией (Ridge)
- линейную регрессию с L1-регуляризацией (Lasso)

Полученное значение RMSE должно быть меньше значения найденного с использованием предсказаний средним значением целевого признака валидационной выборки.

In [9]:
def model_search(name, features_train, target_train,
                       features_valid, target_valid):
    """
    Функция принимает на вход название под которым будет сохранена модель, 
    наборы признаков (features, target) обучающей и валидационной выборок.
    Последовательно перебирает модели линейной регрессии и 
    ищет среди них ту, при которой достигается минимальное 
    значение RMSE на валидационной выборке.
    Сохраняет найденную модель в файл.
    """
    best_model = None
    best_rmse = mean_squared_error(target_valid, 
                                   [target_valid.mean()] * len(target_valid), 
                                   squared=False)
    model = LinearRegression()
    model.fit(features_train, target_train)
    predict = model.predict(features_valid)
    rmse = mean_squared_error(target_valid, predict, squared=False)
    if rmse < best_rmse:
        best_rmse = rmse
        best_model = model
    
    for i in [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]:
        model_ridge = Ridge(alpha=i, max_iter=100000)
        model_ridge.fit(features_train, target_train)
        predict_ridge = model_ridge.predict(features_valid)
        rmse_ridge = mean_squared_error(target_valid, predict_ridge, squared=False)
        if rmse_ridge < best_rmse:
            best_rmse = rmse_ridge
            best_model = model_ridge
    
    for j in [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]:
        model_lasso = Lasso(alpha=j, max_iter=100000)
        model_lasso.fit(features_train, target_train)
        predict_lasso = model_lasso.predict(features_valid)
        rmse_lasso = mean_squared_error(target_valid, predict_lasso, squared=False)
        if rmse_lasso < best_rmse:
            best_rmse = rmse_lasso
            best_model = model_lasso
    
    if best_model:
        # сохраним в файл последнюю найденную модель    
        dump(best_model, "models/" + name + ".joblib")
        print("Модель " + name + " сохранена:" + "\n" + 
              "models/" + name + ".joblib")
        # очистим последнюю найденную модель    
        best_model = None
    else:
        print("Адекватная модель " + name + " не найдена")

- Найдем модели с наименьшим значением RMSE для наших выборок.

In [10]:
for data, name in zip([geo_data_0, geo_data_1, geo_data_2], 
                      ["geo_data_0", "geo_data_1", "geo_data_2"]):
    
    model_search(name, data_split(data)["features_train"],
                       data_split(data)["target_train"],
                       data_split(data)["features_valid"],
                       data_split(data)["target_valid"])

Модель geo_data_0 сохранена:
models/geo_data_0.joblib
Модель geo_data_1 сохранена:
models/geo_data_1.joblib
Модель geo_data_2 сохранена:
models/geo_data_2.joblib


Выведем на экран параметры найденных моделей.

In [11]:
for model, name in zip([load("models/geo_data_0.joblib"), 
                        load("models/geo_data_1.joblib"), 
                        load("models/geo_data_2.joblib")],
                       ["geo_data_0", "geo_data_1", "geo_data_2"]):
    
    print("Модель {}\n{}\nКоэффициенты модели:\n{}\n{}"
          .format(name, 
                  model, 
                  list(np.round(model.coef_, 3)), 
                  " "))

Модель geo_data_0
Lasso(alpha=0.0001, max_iter=100000)
Коэффициенты модели:
[3.593, -14.097, 6.593]
 
Модель geo_data_1
Lasso(alpha=0.0001, max_iter=100000)
Коэффициенты модели:
[-0.145, -0.022, 26.951]
 
Модель geo_data_2
Lasso(alpha=0.1, max_iter=100000)
Коэффициенты модели:
[0.0, -0.009, 5.7]
 


Для всех трех регионов минимум RMSE достигается при использовании L1-регуляризации (штрафуется сумма абсолютных значений коэффициентов). Кроме этого можно видеть как изменяется влияние коэффициентов на предсказания модели от региона к региону. В регионах 1 и 2 повышется влияние коэффициента f2 и снижается влияние коэффициентов f0, f1 по сравнению с нулевым регионом. В регионе 2 коэффициент модели f0 перестает оказывать влияние на предсказания, ключевым становится коэффициент f2. Полученная закономерность может говорить о существенном отличии между регионами.

Выведем на экран величины RMSE для полученных моделей, сравним их с ошибкой, получаемой при прогнозировании с помощью средних значений. Вначале создадим функцию для расчета RMSE и среднего запаса предсказанного сырья на валидационной выборке.

In [12]:
def rmse_calc(data, name, scale=False):
    """
    Функция принимает на вход датафрейм с геоданными, 
    название сохраненной модели, и условие масштабирования.
    Делит датафрейм с помощью функции data_split() или data_scale(),
    в зависимости от условия, и выбирает
    признаки и целевой признак валидационной выборки.
    Выводит словарь, содержащий RMSE и среднее  
    предсказанных значений.
    """
    if not scale:
        features_valid = data_split(data)["features_valid"]
        target_valid = data_split(data)["target_valid"]
        predict = load("models/" + name + ".joblib").predict(features_valid)
        rmse = mean_squared_error(target_valid, predict, squared=False)
    else:
        features_valid = data_scale(data)["features_valid"]
        target_valid = data_scale(data)["target_valid"]
        predict = load("models/" + name + ".joblib").predict(features_valid)
        rmse = mean_squared_error(target_valid, predict, squared=False)
    
    return {"rmse": rmse,
           "mean_predict": np.mean(predict)}

In [13]:
for data, name in zip([geo_data_0, geo_data_1, geo_data_2], 
                      ["geo_data_0", "geo_data_1", "geo_data_2"]):
    
    print("Модель {}\
         \nRMSE модели на валидационной выборке: {:.6f}\
         \nСредний запас предсказанного сырья: {:.6f}\
         \n{}".
         format(name,
                rmse_calc(data, name)["rmse"],
                rmse_calc(data, name)["mean_predict"],
                " "))

Модель geo_data_0         
RMSE модели на валидационной выборке: 37.579421         
Средний запас предсказанного сырья: 92.592570         
 
Модель geo_data_1         
RMSE модели на валидационной выборке: 0.893099         
Средний запас предсказанного сырья: 68.728547         
 
Модель geo_data_2         
RMSE модели на валидационной выборке: 40.029371         
Средний запас предсказанного сырья: 94.965856         
 


Сравним RMSE моделей с ошибкой предсказания, полученной с помощью средних значений целевого признака на валидационной выборке.

In [14]:
for data, name in zip([geo_data_0, geo_data_1, geo_data_2], 
                      ["geo_data_0", "geo_data_1", "geo_data_2"]):

    target_valid = data_split(data)["target_valid"]
    adequacy_rmse = mean_squared_error(target_valid, 
                                       [target_valid.mean()] * len(target_valid), 
                                       squared=False)
    
    print("Проверка модели {} на адекватность\
         \nRMSE с использованием среднего значения\
         \nцелевого признака на валидационной выборке: {:.2f}\
         \nСредний запас сырья на валидационной выборке: {:.2f}\
         \n{}".
         format(name,
                adequacy_rmse,
                np.mean(target_valid),
                " "))

Проверка модели geo_data_0 на адекватность         
RMSE с использованием среднего значения         
целевого признака на валидационной выборке: 44.29         
Средний запас сырья на валидационной выборке: 92.08         
 
Проверка модели geo_data_1 на адекватность         
RMSE с использованием среднего значения         
целевого признака на валидационной выборке: 46.02         
Средний запас сырья на валидационной выборке: 68.72         
 
Проверка модели geo_data_2 на адекватность         
RMSE с использованием среднего значения         
целевого признака на валидационной выборке: 44.90         
Средний запас сырья на валидационной выборке: 94.88         
 


Можно видеть, что найденные модели (за исключением geo_data_1) имеют показатель RMSE близкий к тому, который можно получить с использованием среднего значения на валидационной выборке. Также можно видеть, что найденные модели довольно точно предсказывают средний запас сырья.

Попробуем улучшить качество моделей с помощью масштабирования признаков.

- Создадим копии выборок, содержащих признаки и приведем их к одному масштабу для дальнейших экспериментов с моделями машинного обучения. Для этого создадим функцию для масштабирования признаков.

In [15]:
def data_scale(data):
    """
    Функция принимает на вход датасет с информацией о месторождениях.
    Делит его с помощью функции data_split(), создает копии обучающей и 
    валидационной выборок. После этого масштабирует признаки с 
    использованием StandardScaler(), целевой признак оставляет без
    изменения.
    Выводит словарь со значениями признаков после масштабирования и
    целевого признака без изменения.
    """
    numeric = ["f0", "f1", "f2"]
    features_train_scale = data_split(data)["features_train"].copy()
    target_train_scale = data_split(data)["target_train"].copy()
    features_valid_scale = data_split(data)["features_valid"].copy()
    target_valid_scale = data_split(data)["target_valid"].copy()
    
    scaler = StandardScaler()
    scaler.fit(features_train_scale[numeric])
    
    features_train_scale[numeric] = scaler.transform(features_train_scale[numeric])
    features_valid_scale[numeric] = scaler.transform(features_valid_scale[numeric])
    
    return {"features_train": features_train_scale,
            "target_train": target_train_scale,
            "features_valid": features_valid_scale,
            "target_valid": target_valid_scale}

Проверим работу функции. Выведем на экран данные geo_data_0 до и после масштабирования.

In [16]:
for features in ["features_train", "target_train", "features_valid", "target_valid"]:
    print("Выборка", features, "до масштабирования")
    display(data_split(geo_data_0)[features].head(2))
    print("Выборка", features, "после масштабирования")
    display(data_scale(geo_data_0)[features].head(2))
    print(" ")

Выборка features_train до масштабирования


,f0,f1,f2
27212,0.022450,0.951034,2.197333
7866,1.766731,0.007835,6.436602


Выборка features_train после масштабирования


,f0,f1,f2
27212,-0.544828,1.390264,-0.094959
7866,1.455912,-0.480422,1.209567


 
Выборка target_train до масштабирования


...

,f0,f1,f2
71751,0.517917,-0.610097,-0.126226
80493,0.568391,-0.086063,-0.814914


 
Выборка target_valid до масштабирования


71751     10.038645
80493    114.551489
Name: product, dtype: float64

Выборка target_valid после масштабирования


71751     10.038645
80493    114.551489
Name: product, dtype: float64

Как и планировалось происходит масштабирование признаков и не происходит масштабирование целевого признака.

- Найдем модели с наименьшим значением RMSE для выборок после масштабирования признаков.

In [17]:
for data, name in zip([geo_data_0, geo_data_1, geo_data_2], 
                      ["geo_data_scale_0", "geo_data_scale_1", "geo_data_scale_2"]):
    
    model_search(name, data_scale(data)["features_train"],
                       data_scale(data)["target_train"],
                       data_scale(data)["features_valid"],
                       data_scale(data)["target_valid"])

Модель geo_data_scale_0 сохранена:
models/geo_data_scale_0.joblib
Модель geo_data_scale_1 сохранена:
models/geo_data_scale_1.joblib
Модель geo_data_scale_2 сохранена:
models/geo_data_scale_2.joblib


Выведем на экран параметры найденных моделей.

In [18]:
for model, name in zip([load("models/geo_data_scale_0.joblib"), 
                        load("models/geo_data_scale_1.joblib"), 
                        load("models/geo_data_scale_2.joblib")],
                       ["geo_data_scale_0", "geo_data_scale_1", "geo_data_scale_2"]):
    
    print("Модель {}\n{}\nКоэффициенты модели:\n{}\n{}"
          .format(name, 
                  model, 
                  list(np.round(model.coef_, 3)), 
                  " "))

Модель geo_data_scale_0
Lasso(alpha=0.0001, max_iter=100000)
Коэффициенты модели:
[3.132, -7.108, 21.425]
 
Модель geo_data_scale_1
Lasso(alpha=0.001, max_iter=100000)
Коэффициенты модели:
[-1.299, -0.112, 45.885]
 
Модель geo_data_scale_2
Lasso(alpha=0.01, max_iter=100000)
Коэффициенты модели:
[0.044, -0.063, 19.81]
 


После масштабирования признаков параметры моделей заметно изменились. В частности, можно отметить увеличение коэффициента регуляризации для первого региона, что говорит о большем штрафе за использование высоких значений коэффициентов (f2) и повышении влияния f0 и f1 в этой модели. В модели для второго региона, наоборот, произошло снижение коэффициента alpha, что также привело к увеличению вляния признаков f0, f1. В модели для первого региона коэффициент регуляризации остался без изменения, но повысилось влияние признака f3 и снизилось влияние f2 на результаты предсказаний.

Выведем на экран RMSE моделей после масштабирования признаков (на валидационной выборке) и средний запас предсказанного сырья для кажого региона.

In [19]:
for data, name in zip([geo_data_0, geo_data_1, geo_data_2], 
                      ["geo_data_scale_0", "geo_data_scale_1", "geo_data_scale_2"]):
    
    print("Модель {}\
         \nRMSE модели на валидационной выборке: {:.6f}\
         \nСредний запас предсказанного сырья: {:.6f}\
         \n{}".
         format(name,
                rmse_calc(data, name, scale=True)["rmse"],
                rmse_calc(data, name, scale=True)["mean_predict"],
                " "))

Модель geo_data_scale_0         
RMSE модели на валидационной выборке: 37.579421         
Средний запас предсказанного сырья: 92.592570         
 
Модель geo_data_scale_1         
RMSE модели на валидационной выборке: 0.893093         
Средний запас предсказанного сырья: 68.728547         
 
Модель geo_data_scale_2         
RMSE модели на валидационной выборке: 40.029692         
Средний запас предсказанного сырья: 94.965209         
 


Можно видеть, что масштабирование признаков не привело к улучшению метрик модели, кроме того значения RMSE и среднего запаса сырья совпадают вплоть до 6 знака после запятой. Это говорит о пределе возможностей линейной модели и эффективности L1-регуляризации на исходных данных. Дальнейшую работу для удобства будем проводить на данных без масштабирования.

- Сохраним предсказания и правильные ответы на валидационной выборке.
- Изменим тип предсказанных данных с np.array на pd.Series и присвоим им индексы такие же как у целевого признака.

In [20]:
target_valid_geo_data_0 = data_split(geo_data_0)["target_valid"]
predict_valid_geo_data_0 = load("models/geo_data_0.joblib")\
                           .predict(data_split(geo_data_0)["features_valid"])
predict_valid_geo_data_0 = pd.Series(predict_valid_geo_data_0)
predict_valid_geo_data_0.index = target_valid_geo_data_0.index

target_valid_geo_data_1 = data_split(geo_data_1)["target_valid"]
predict_valid_geo_data_1 = load("models/geo_data_1.joblib")\
                           .predict(data_split(geo_data_1)["features_valid"])
predict_valid_geo_data_1 = pd.Series(predict_valid_geo_data_1)
predict_valid_geo_data_1.index = target_valid_geo_data_1.index

target_valid_geo_data_2 = data_split(geo_data_2)["target_valid"]
predict_valid_geo_data_2 = load("models/geo_data_2.joblib")\
                           .predict(data_split(geo_data_2)["features_valid"])
predict_valid_geo_data_2 = pd.Series(predict_valid_geo_data_2)
predict_valid_geo_data_2.index = target_valid_geo_data_2.index

С помощью sweetviz проанализируем как изменяется величина остатков (predict - target) для каждого региона.

In [ ]:
valid_geo_data_0 = data_split(geo_data_0)["features_valid"]
valid_geo_data_0["difference"] = predict_valid_geo_data_0 - target_valid_geo_data_0

report_valid_0 = sv.analyze([valid_geo_data_0, "valid_geo_data_0"], 
                            target_feat="difference")
report_valid_0.show_html("sweetviz/valid_geo_data_0.html")
report_valid_0.show_notebook()

In [ ]:
valid_geo_data_1 = data_split(geo_data_1)["features_valid"]
valid_geo_data_1["difference"] = predict_valid_geo_data_1 - target_valid_geo_data_1

report_valid_1 = sv.analyze([valid_geo_data_1, "valid_geo_data_1"], 
                            target_feat="difference")
report_valid_1.show_html("sweetviz/valid_geo_data_1.html")
report_valid_1.show_notebook()

In [ ]:
valid_geo_data_2 = data_split(geo_data_2)["features_valid"]
valid_geo_data_2["difference"] = predict_valid_geo_data_2 - target_valid_geo_data_2

report_valid_2 = sv.analyze([valid_geo_data_2, "valid_geo_data_2"], 
                            target_feat="difference")
report_valid_2.show_html("sweetviz/valid_geo_data_2.html")
report_valid_2.show_notebook()

Результаты показывают, что остатки во всех трех регионах распределены по нормальному закону. Их минимальный размах в регионе 1, намного больший в регионах 0 и 2, что согласуется с найденными ранее величинами RMSE. Интересно отметить, что наибольшая абсолютная величина разности между предсказанными и истинными значениями в регионе 0 наблюдается при экстремальных значениях признака f2, причем модель завышает результаты при максимальных значениях признака и занижает при минимальных. В регионе 2 наибольшая абсолютная величина остатков наблюдается при экстремальных значениях всех трех признаков. Причем для признаков f0, f1 наблюдаются заниженные предсказанные величины, а для признака f2 также как и в регионе 0 наблюдается завышение результатов при максимальных значениях признака f2 и занижение при минимальных. Можно заключить, что найденные модели машинного обучения, для некоторых комбинаций признаков, будут завышать величину запасов предсказанного сырья.

## Подготовка к расчёту прибыли

- Сохраним все ключевые значения для расчётов в отдельных переменных.

In [24]:
# количество лучших скважин, выбранных с помощью машинного обучения
BEST_NUM = 200
# бюджет на разработку скважин в регионе
BUDGET_REGION = 10e9
# доход с каждой единицы продукта
INCOME_UNIT = 450e3

- Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины. Сравним полученный объём сырья со средним запасом в каждом регионе.

In [25]:
required_volume = BUDGET_REGION / (BEST_NUM * INCOME_UNIT)
print("Средний объем сырья для безубыточной разработки:", 
      round(required_volume, 1), sep="\n")

Средний объем сырья для безубыточной разработки:
111.1


Можно видеть, что при заданных ценах на баррель сырья и бюджете на разработку скважин в регионе средний объем сырья для 200 скважин должен составлять 111 баррелей. Это заметно больше среднего содержания сырья для исследуемых регионов, в которых данная величина составляет от 68 (регион 1) до 95 (регионы 0 и 2) баррелей. По этой причине для разработки необходимо выбирать скважины с максимальным запасом.

- Напишем функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели.

In [26]:
def profit_calc(true, predict, best_num=BEST_NUM, 
                               income_unit=INCOME_UNIT, 
                               budget_region=BUDGET_REGION):
    """
    Функция примает на вход Series с истинными и предсказанными
    значениями прибыли (требуется совпадение индексов), 
    количество лучших анализируемых скважин, доход с каждой единицы 
    продукта, бюджет на разработку скважин в регионе.
    Выбирает скважины с максимальными значениями предсказаний, 
    суммирует целевое значение объема сырья, соответствующее этим 
    предсказаниям.
    Выводит прибыль для полученного объема сырья.
    """
    if len(predict) >= best_num:
        predict = predict.sort_values(ascending=False).iloc[:best_num]
        true = true[predict.index]
        revenue = true.sum() * income_unit
        profit = revenue - budget_region
        return profit
    else:
        print("Ошибка! Размер выборки меньше необходимого")

## Расчёт прибыли и рисков 

- Применим технику Bootstrap с 1000 выборок и объемом каждой выборки в 500 единиц, чтобы найти распределение прибыли. Выборки будем создавать без возвращения значений, т.к. при разработке региона исследуется 500 разных точек. 
- Найдем среднюю прибыль, 95%-й доверительный интервал и риск убытков (отрицательной прибыли).

In [27]:
# количество выборок
bootstrap_number = 1000
# объем каждой выборки в единицах
bootstrap_volume = 500
# доверительный интервал для средней прибыли
quantile_bottom = 0.025
quantile_upper = 0.975

# списки для хранения величины прибыли
profit_list_geo_data_0 = list()
profit_list_geo_data_1 = list()
profit_list_geo_data_2 = list()

# проведем Bootstrap
state = np.random.RandomState(12345)
for predict_valid,\
    target_valid, profit_list in zip([predict_valid_geo_data_0, 
                                      predict_valid_geo_data_1, 
                                      predict_valid_geo_data_2], 
                                     [target_valid_geo_data_0, 
                                      target_valid_geo_data_1, 
                                      target_valid_geo_data_2],
                                     [profit_list_geo_data_0,
                                      profit_list_geo_data_1,
                                      profit_list_geo_data_2]):
    
    for _ in range(bootstrap_number):
        bootstrap_predict = predict_valid.sample(n=bootstrap_volume, replace=False, 
                                                 random_state=state)
        
        predict = bootstrap_predict
        true = target_valid[predict.index]
        
        profit = profit_calc(true, predict)
        profit_list.append(profit)

# расчитаем величину прибыли
for profit_list, region in zip([profit_list_geo_data_0,
                                profit_list_geo_data_1,
                                profit_list_geo_data_2],
                               ["geo_data_0", "geo_data_1", "geo_data_2"]):
    print("Регион {}\
         \nСредняя прибыль: {:.1f}\
         \n{} %-й доверительный интервал: [{:.1f}, {:.1f}]\
         \nРиск убытков: {:.1f}%"
    .format(region,
    # средняя прибыль        
            pd.Series(profit_list).mean(),
    # доверительный интервал
            (quantile_upper - quantile_bottom) * 100, 
            pd.Series(profit_list).quantile(quantile_bottom),
            pd.Series(profit_list).quantile(quantile_upper),
    # риск убытков
            ((pd.Series(profit_list) < 0).sum() / 
             len(pd.Series(profit_list_geo_data_0))) * 100))

Регион geo_data_0         
Средняя прибыль: 380710890.7         
95.0 %-й доверительный интервал: [-126947638.0, 879613967.8]         
Риск убытков: 7.2%
Регион geo_data_1         
Средняя прибыль: 454785434.8         
95.0 %-й доверительный интервал: [46730084.8, 840213356.3]         
Риск убытков: 1.3%
Регион geo_data_2         
Средняя прибыль: 388916252.1         
95.0 %-й доверительный интервал: [-115609565.8, 905292904.4]         
Риск убытков: 7.3%


Условие по допустимой вероятности убытков (ниже 2,5 %) проходит только регион geo_data_1.

## Вывод
Можно заключить, что наиболее удачным регионом для разработки скважин является geo_data_1. Этот регион обладает наиболее предсказуемыми запасами сырья, которые удается определить с наибольшей точностью, используя линейную модель. В нем наблюдается одновременно наименьший риск убытков и наибольшее значение средней прибыли. Найденный доверительный интервал для этого региона показывает, что с вероятностью 95% можно получить ненулевую прибыль.